# Transforming Inventory Data

Imagine your data looks like:

In [1]:
%cat transforming_inventory_data/inventory/hosts.yaml

---
rtr00:
    data:
        user: automation_user
rtr01:
    data:
        user: automation_user


And we want to do two things:

1. Map ``user`` to ``username``
2. Prompt the user for the password and add it

You can easily do that using a transform_function. For instance:

## Modifying hosts' data

You can modify inventory data (regardless of the plugin you are using) on the fly easily by password a ``transform_function`` like this:

In [2]:
from nornir import InitNornir
import pprint

def adapt_host_data(host):
    # This function receives a Host object for manipulation
    host.username = host["user"]

nr = InitNornir(
    inventory={
            "plugin": "nornir.plugins.inventory.simple.SimpleInventory",
            "options": {
                "host_file": "transforming_inventory_data/inventory/hosts.yaml",
                "group_file": "transforming_inventory_data/inventory/groups.yaml",
            },
            "transform_function": adapt_host_data,
    },
)
for name, host in nr.inventory.hosts.items():
    print(f"{name}.username: {host.username}")

rtr00.username: automation_user
rtr01.username: automation_user


### Specifying the transform function in the configuration

You can also specify the transform function in the configuration. In order for that to work the function must be importable. For instance:

In [3]:
%cat transforming_inventory_data/helpers.py

def adapt_host_data(host):
    # This function receives a Host object for manipulation
    host.username = host["user"]


Now let's verify we can, indeed, import the function:

In [4]:
from transforming_inventory_data.helpers import adapt_host_data
adapt_host_data

<function transforming_inventory_data.helpers.adapt_host_data(host)>

Now everything we have to do is put the import path as the ``transform_function`` configuration option:

In [5]:
%cat transforming_inventory_data/config.yaml

---
inventory:
    plugin: nornir.plugins.inventory.simple.SimpleInventory
    options:
        host_file: "transforming_inventory_data/inventory/hosts.yaml"
        group_file: "transforming_inventory_data/inventory/groups.yaml"
    transform_function: "transforming_inventory_data.helpers.adapt_host_data"


And then we won't need to specify it when calling ``InitNornir``:

In [6]:
from nornir import InitNornir
import pprint

nr = InitNornir(
    config_file="transforming_inventory_data/config.yaml",
)
for name, host in nr.inventory.hosts.items():
    print(f"{name}.username: {host.username}")

rtr00.username: automation_user
rtr01.username: automation_user


## Setting a default password

You might be in a situation where you may want to prompt the user for some information, for instance, a password. You can leverage the ``defaults`` for something like this (although you could just put in under the hosts themselves or the groups). Let's see an example:

In [7]:
from nornir import InitNornir

# let's pretend we used raw_input or something like that
# password = raw_input("Please, enter password: ")
password = "a_secret_password"

nr = InitNornir(
    config_file="transforming_inventory_data/config.yaml",
)
print("Before setting password: ", nr.inventory.hosts["rtr00"].password)
nr.inventory.defaults.password = password
print("After setting password: ", nr.inventory.hosts["rtr00"].password)
print("After setting password: ", nr.inventory.defaults.password)

Before setting password:  None
After setting password:  a_secret_password
After setting password:  a_secret_password


For more information on how inheritance works check the tutorial section [inheritance model](../tutorials/intro/inventory.rst)

## Passing options to the transform function

You can also pass options to the transform function, like in this example:


In [ ]:
from nornir import InitNornir


def adapt_host_data(host, password, something_else):
    host.data["password"] = password

    
options = {
    "password": "a_secret_password",
    "something_else": "unused"
}
    
nr = InitNornir(
    inventory={
        "plugin": "nornir.plugins.inventory.simple.SimpleInventory",
        "options": {
            "host_file": "transforming_inventory_data/inventory/hosts.yaml",
            "group_file": "transforming_inventory_data/inventory/groups.yaml",
        },
        "transform_function": adapt_host_data,
        "transform_function_options": options,
    }
)
for name, host in nr.inventory.hosts.items():
    print(f"{name}.password: {host.password}")